<a href="https://colab.research.google.com/github/YahiaAbdeldjalilBenyahia/graph-coloring/blob/main/exact/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import numpy as np
import requests

In [ ]:
DEBUG = True

def debug_print(*args, **kwargs):
    if DEBUG:
        print(*args, **kwargs)

In [ ]:
def lire_graphe_triangulaire(url_fichier):
    """
    Lit un fichier DIMACS et convertit le graphe en une matrice triangulaire inférieure.

    :param url_fichier: URL du fichier .col contenant le graphe
    :return: Matrice triangulaire inférieure sous forme de liste de listes
    """
    with open(url_fichier, 'r') as f:
      # Lit toutes les lignes et les stocke dans une liste lignes.
      lignes = f.readlines()

    nb_sommets = 0
    aretes = []

    for ligne in lignes:
        ligne = ligne.strip()
        if ligne.startswith('p'):
            _, _, nb_sommets, _ = ligne.split()
            nb_sommets = int(nb_sommets)
        elif ligne.startswith('e'):
            _, sommet1, sommet2 = ligne.split()
            sommet1, sommet2 = int(sommet1), int(sommet2)
            aretes.append((sommet1, sommet2))

    # Création de la matrice triangulaire inférieure
    matrice_triangulaire = [[0] * (i + 1) for i in range(nb_sommets)]
    for sommet1, sommet2 in aretes:
        sommet1 -= 1  # Ajustement des indices (DIMACS commence à 1)
        sommet2 -= 1
        if sommet1 > sommet2:
            matrice_triangulaire[sommet1][sommet2] = 1
        else:
            matrice_triangulaire[sommet2][sommet1] = 1

    return matrice_triangulaire

In [ ]:
def greedy_coloring(matrice_triangulaire):
    """
    Applique un algorithme glouton pour obtenir une solution initiale de coloration.
    Cette solution sert de borne supérieure initiale (upper bound) pour le branch and bound.

    :param matrice_triangulaire: La matrice triangulaire inférieure représentant le graphe.
    :return: Tuple (nombre de couleurs utilisées, liste de coloration)
    """
    n = len(matrice_triangulaire)
    colors = [None] * n
    for vertex in range(n):
        forbidden = set()
        for j in range(n):
            if colors[j] is not None:
                if vertex > j and matrice_triangulaire[vertex][j] == 1:
                    forbidden.add(colors[j])
                elif vertex < j and matrice_triangulaire[j][vertex] == 1:
                    forbidden.add(colors[j])
        color = 1
        while color in forbidden:
            color += 1
        colors[vertex] = color
    return len(set(colors)), colors

In [ ]:
def build_neighbor_list(matrice_triangulaire):
    """
    Pré-calcule la liste des voisins pour chaque sommet à partir de la matrice triangulaire.

    :param matrice_triangulaire: Matrice triangulaire inférieure représentant le graphe.
    :return: Liste de listes, où chaque sous-liste contient les indices des voisins du sommet.
    """
    n = len(matrice_triangulaire)
    neighbors = [[] for _ in range(n)]
    for i in range(n):
        for j in range(i):
            if matrice_triangulaire[i][j] == 1:
                neighbors[i].append(j)
                neighbors[j].append(i)
    return neighbors

In [ ]:
def select_next_vertex_dsatur(assignment, uncolored, neighbors):
    """
    Sélectionne dynamiquement le prochain sommet à colorier selon la règle DSATUR.
    Le sommet choisi est celui qui a la plus grande "saturation" (nombre de couleurs distinctes parmi ses voisins coloriés)
    et, en cas d'égalité, celui avec le degré le plus élevé.

    :param assignment: Liste des couleurs assignées (None si non colorié)
    :param uncolored: Ensemble des indices de sommets non colorés
    :param neighbors: Liste pré-calculée des voisins pour chaque sommet
    :return: L'indice du sommet à colorier ensuite
    """
    best_vertex = None
    best_sat = -1
    best_degree = -1
    for v in uncolored:
        sat = len({assignment[u] for u in neighbors[v] if assignment[u] is not None})
        degree = len(neighbors[v])
        if sat > best_sat or (sat == best_sat and degree > best_degree):
            best_sat = sat
            best_degree = degree
            best_vertex = v
    return best_vertex

In [ ]:
def branch_and_bound_coloring_dsatur(matrice_triangulaire):
    """
    Implémente le branch and bound pour la coloration de graphe en utilisant DSATUR pour le choix du prochain sommet.

    Optimisations appliquées :
      - Utilisation d'une heuristique gloutonne pour obtenir une solution initiale (borne supérieure).
      - Pré-calcul des voisins pour accélérer la vérification des conflits.
      - Sélection dynamique du prochain sommet à colorier par DSATUR.
      - Branching sur les couleurs déjà utilisées + une nouvelle couleur.

    Chaque état est représenté par :
      - assignment : liste des couleurs assignées (None si non colorié)
      - uncolored : ensemble des indices des sommets non colorés
      - num_colors_used : nombre de couleurs utilisées jusqu'ici

    :return: Tuple (nombre chromatique optimal, solution de coloration, temps d'exécution)
    """
    start_time = time.time()
    n = len(matrice_triangulaire)

    # Obtenir une solution initiale par heuristique gloutonne
    initial_color_count, initial_solution = greedy_coloring(matrice_triangulaire)
    debug_print(f"Initial heuristic solution: {initial_solution} using {initial_color_count} colors.")

    best_color_count = initial_color_count
    best_solution = initial_solution.copy()

    # Pré-calculer la liste des voisins
    neighbors = build_neighbor_list(matrice_triangulaire)

    # État initial : aucun sommet colorié
    initial_assignment = [None] * n
    uncolored = set(range(n))
    initial_state = (initial_assignment, uncolored, 0)

    stack = [initial_state]
    nodes_processed = 0  # Compteur de noeuds explorés

    while stack:
        nodes_processed += 1
        assignment, uncolored, num_colors_used = stack.pop()

        # Affichage périodique de la progression
        if nodes_processed % 1000 == 0:
            print(f"Nodes processed: {nodes_processed}, Current best: {best_color_count}, Stack size: {len(stack)}")

        if num_colors_used >= best_color_count:
            continue

        # Si tous les sommets sont coloriés, mise à jour de la meilleure solution.
        if not uncolored:
            used_colors = len({color for color in assignment if color is not None})
            debug_print(f"Complete solution found with {used_colors} colors: {assignment}")
            if used_colors < best_color_count:
                best_color_count = used_colors
                best_solution = assignment.copy()
                debug_print(f"New best solution with {best_color_count} colors!")
            continue

        # Sélectionner le prochain sommet à colorier par DSATUR
        v = select_next_vertex_dsatur(assignment, uncolored, neighbors)
        forbidden = {assignment[u] for u in neighbors[v] if assignment[u] is not None}

        # Déterminer les couleurs disponibles: celles déjà utilisées non interdites et une nouvelle couleur.
        available_colors = [color for color in range(1, num_colors_used + 1) if color not in forbidden]
        available_colors.append(num_colors_used + 1)

        for color in available_colors:
            new_assignment = assignment.copy()
            new_assignment[v] = color
            new_num_colors_used = max(num_colors_used, color)

            # Pruning: Élagage des branches qui ne peuvent pas améliorer la solution.
            if new_num_colors_used >= best_color_count:
                debug_print(f"Elagage de la branche dans le noeud {v} avec la couleur {color} (utilise: {new_num_colors_used} >= meilleur: {best_color_count})")
                continue

            new_uncolored = uncolored.copy()
            new_uncolored.remove(v)
            stack.append((new_assignment, new_uncolored, new_num_colors_used))

    execution_time = time.time() - start_time
    print(f"Nombre total des noeuds: {nodes_processed}")
    return best_color_count, best_solution, execution_time

In [ ]:
def reduce_to_first_n_nodes(matrix, n):
    # On prend les n premières lignes de la matrice (en faisant une copie profonde)
    return [row[:i+1] for i, row in enumerate(matrix[:n])]

In [ ]:
def reduire_dimacs(input_file, output_file, n_noeuds):
    """
    Réduit un fichier DIMACS à un sous-graphe composé des n premiers noeuds.

    Cette fonction lit le fichier DIMACS d'entrée, filtre les arêtes dont
    les deux extrémités sont comprises dans [1, n_noeuds], met à jour l'en-tête
    avec le nouveau nombre de noeuds et d'arêtes, et écrit le résultat dans
    le fichier DIMACS de sortie.

    :param input_file: Chemin vers le fichier DIMACS original.
    :param output_file: Chemin vers le fichier DIMACS réduit à créer.
    :param n_noeuds: Nombre de noeuds à retenir (les noeuds 1 à n_noeuds).
    """
    # Lecture du fichier original
    with open(input_file, 'r') as f:
        lignes = f.readlines()

    # Listes pour stocker les lignes de commentaires et les arêtes retenues.
    commentaires = []
    aretes_retenues = []

    for ligne in lignes:
        # Conserver toutes les lignes de commentaires
        if ligne.startswith("c"):
            commentaires.append(ligne)
        # On ignore la ligne d'en-tête d'origine (commençant par "p") car nous la recalculerons.
        elif ligne.startswith("p"):
            pass
        # Traiter les lignes d'arêtes
        elif ligne.startswith("e"):
            parts = ligne.strip().split()
            if len(parts) < 3:
                continue
            try:
                u = int(parts[1])
                v = int(parts[2])
            except ValueError:
                continue
            # Conserver l'arête seulement si les deux noeuds sont dans [1, n_noeuds]
            if u <= n_noeuds and v <= n_noeuds:
                aretes_retenues.append(ligne)

    # Création de la nouvelle ligne d'en-tête
    nouvelle_entete = f"p edge {n_noeuds} {len(aretes_retenues)}\n"

    # Écriture du nouveau fichier DIMACS
    with open(output_file, 'w') as f:
        # Ecriture des commentaires existants
        for commentaire in commentaires:
            f.write(commentaire)
        # Ajouter un commentaire sur la réduction
        f.write(f"c Fichier DIMACS réduit aux {n_noeuds} premiers noeuds.\n")
        # Ecriture de la nouvelle ligne d'en-tête
        f.write(nouvelle_entete)
        # Ecriture de toutes les arêtes retenues
        for arete in aretes_retenues:
            f.write(arete)


In [ ]:
reduire_dimacs("dsjc250.1.col", "dsjc155.1.col", 155)

In [ ]:
url_fichier = "dsjc152.1.col"
matrice = lire_graphe_triangulaire(url_fichier)
# n = 14  # nombre de nœuds à garder
# reduced_matrix = reduce_to_first_n_nodes(matrice, n)
nb_chromatic, solution, temps = branch_and_bound_coloring_dsatur(matrice)
print("\n==========================")
print("Nombre chromatique optimal:", nb_chromatic)
print("Solution de coloration (par sommet):", solution)
print("Temps d'exécution: {:.4f} secondes".format(temps))

## Isolated graph

In [ ]:
# Number of nodes
n = 200

# Create a lower triangular matrix filled with zeros
isolated_graph_lower_triangular = np.zeros((n, n), dtype=int)

# Convert to list of lists if needed
isolated_graph_matrix_list = isolated_graph_lower_triangular.tolist()
# print(isolated_graph_matrix_list)
# Print only the lower triangular part
g = []
for i in range(n):
    g.append(isolated_graph_matrix_list[i][:i+1])

In [ ]:
nb_chromatic, solution, temps = branch_and_bound_coloring_dsatur(g)
print("\n==========================")
print("Nombre chromatique optimal:", nb_chromatic)
print("Solution de coloration (par sommet):", solution)
print("Temps d'exécution: {:.4f} secondes".format(temps))

Initial heuristic solution: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] using 1 colors.
Elagage de la branche dans le noeud 0 avec la couleur 1 (utilise: 1 >= meilleur: 1)
Nombre total des noeuds: 1

Nombre chromatique optimal: 1
Solution de coloration (par sommet): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1